In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
import joblib


In [2]:
# Fungsi untuk ekstraksi fitur dari data sinyal
def extract_features(data):
    # Hitung rata-rata, nilai maksimum, nilai minimum, dan standar deviasi dari data
    mean_value = np.mean(data)
    max_value = np.max(data)
    min_value = np.min(data)
    std_value = np.std(data)
    
    return [mean_value, max_value, min_value, std_value]

# List untuk menyimpan nilai fitur dan kelas data
features_list = []
class_labels = []

# Daftar folder yang berisi data untuk masing-masing kelas
folder_paths = ['E:/LSTM/4_class/class1/', 
                'E:/LSTM/4_class/class2/', 
                'E:/LSTM/4_class/class3/', 
                'E:/LSTM/4_class/class4/']

In [3]:
# Loop untuk membaca dan memproses data dari folder
for class_idx, folder_path in enumerate(folder_paths):
    file_list = os.listdir(folder_path)
    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        data = df['Y'].values
        
        # Ekstraksi fitur dari data sinyal
        features = extract_features(data)
        features_list.append(features)
        
        # Tambahkan kelas (label) ke dalam list
        class_label = f'class{class_idx + 1}'
        class_labels.append(class_label)


In [4]:
# Ubah label kelas menjadi angka dengan LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(class_labels)
y_encoded = to_categorical(y)
num_classes = len(label_encoder.classes_)

# Ubah fitur menjadi bentuk array numpy
X = np.array(features_list)

# Pisahkan data menjadi data training dan data testing
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Konstruksi model LSTM
model = Sequential()
model.add(Dense(64, input_shape=(X_train.shape[1],)))
model.add(Dense(num_classes, activation='softmax'))

In [5]:
# Kompilasi model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
2/2 [==============================] - 2s 711ms/step - loss: 1.4386 - accuracy: 0.1875 - val_loss: 1.3015 - val_accuracy: 0.4375
Epoch 2/50
2/2 [==============================] - 0s 61ms/step - loss: 1.4220 - accuracy: 0.1875 - val_loss: 1.3150 - val_accuracy: 0.4375
Epoch 3/50
2/2 [==============================] - 0s 53ms/step - loss: 1.4090 - accuracy: 0.1875 - val_loss: 1.3245 - val_accuracy: 0.4375
Epoch 4/50
2/2 [==============================] - 0s 61ms/step - loss: 1.3992 - accuracy: 0.2031 - val_loss: 1.3356 - val_accuracy: 0.5000
Epoch 5/50
2/2 [==============================] - 0s 67ms/step - loss: 1.3903 - accuracy: 0.2031 - val_loss: 1.3462 - val_accuracy: 0.5000
Epoch 6/50
2/2 [==============================] - 0s 79ms/step - loss: 1.3853 - accuracy: 0.2500 - val_loss: 1.3576 - val_accuracy: 0.5000
Epoch 7/50
2/2 [==============================] - 0s 56ms/step - loss: 1.3808 - accuracy: 0.2656 - val_loss: 1.3699 - val_accuracy: 0.3750
Epoch 8/50
2/2 [==========

In [6]:
# Simpan model ke file menggunakan joblib
model_filename = 'signal_classifier_model_lstm.joblib'
joblib.dump(model, model_filename)

['signal_classifier_model_lstm.joblib']

In [7]:
# Contoh prediksi pada data baru
# Ganti "new_data" dengan data fitur baru yang ingin diprediksi
new_data = np.array([[mean_value_new, max_value_new, min_value_new, std_value_new]])
predicted_class = model.predict_classes(new_data)
print("Predicted Class:", label_encoder.inverse_transform(predicted_class)[0])

NameError: name 'mean_value_new' is not defined

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

# Fungsi untuk ekstraksi fitur highest peak dari PSD
def extract_highest_peak_psd(data, fs, lowcut, highcut, order):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    
    #Filter bandpass
    filtered_signal = signal.filtfilt(b, a, data)
    
    # Rectify signal
    rectified_signal = np.abs(filtered_signal)
    
    # One-sided FFT
    signal_fft = np.fft.fft(filtered_signal)
    signal_freq = np.fft.fftfreq(len(filtered_signal), 1/fs)
    n = len(filtered_signal)
    freq = signal_freq[0:int(n/2)]
    fft_amp = np.abs(signal_fft)[0:int(n/2)]/n*2
    
    # One-sided PSD
    psd = (1/(fs*n)) * np.square(np.abs(signal_fft[:n//2]))
    freq_psd = signal_freq[0:int(n/2)]
    psd_one_side = psd[0:int(n/2)]
    
    # Ekstraksi fitur highest peak dari PSD
    highest_peak_idx = np.argmax(psd_one_side)
    highest_peak_value = psd_one_side[highest_peak_idx]
    return highest_peak_value

# Fungsi untuk membaca dataset dari folder kelas
def read_data_from_folder(folder_path, class_label, fs, lowcut, highcut, order):
    dataframes = []
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        start_index = 1
        end_index = 1551
        df = df[start_index:end_index]
        data = df['Y'].values
        highest_peak_psd = extract_highest_peak_psd(data, fs, lowcut, highcut, order)
        df_highest_peak_psd = pd.DataFrame({'Highest Peak PSD': [highest_peak_psd], 'Class': [class_label]})
        dataframes.append(df_highest_peak_psd)
    return pd.concat(dataframes, ignore_index=True)

# Path ke folder yang berisi dataset untuk masing-masing kelas
class1_folder = 'E:/LSTM/4_class/class1/'
class2_folder = 'E:/LSTM/4_class/class2/'
class3_folder = 'E:/LSTM/4_class/class3/'
class4_folder = 'E:/LSTM/4_class/class4/'

# Parameter filter
fs = 1000  # Frekuensi sampling
lowcut = 1.0  # Frekuensi cutoff rendah untuk filter bandpass
highcut = 50.0  # Frekuensi cutoff tinggi untuk filter bandpass
order = 4  # Order filter

# Membaca data untuk masing-masing kelas
class1_data = read_data_from_folder(class1_folder, class_label=0, fs=fs, lowcut=lowcut, highcut=highcut, order=order)
class2_data = read_data_from_folder(class2_folder, class_label=1, fs=fs, lowcut=lowcut, highcut=highcut, order=order)
class3_data = read_data_from_folder(class3_folder, class_label=2, fs=fs, lowcut=lowcut, highcut=highcut, order=order)
class4_data = read_data_from_folder(class4_folder, class_label=3, fs=fs, lowcut=lowcut, highcut=highcut, order=order)

# Menggabungkan data dari masing-masing kelas menjadi satu DataFrame utama
combined_data = pd.concat([class1_data, class2_data, class3_data, class4_data], ignore_index=True)

# Memisahkan fitur highest peak PSD dan label kelas
X = np.array(combined_data['Highest Peak PSD'].values.tolist())
y = np.array(combined_data['Class'].values)

# One-hot encode label kelas
y_encoded = to_categorical(y, num_classes=4)

# Memisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Mengubah bentuk data untuk LSTM (tambahkan dimensi waktu)
X_train_lstm = np.expand_dims(X_train, axis=1)
X_test_lstm = np.expand_dims(X_test, axis=1)

# Membuat model LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(X_train_lstm.shape[1], 1)))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Menyimpan model terbaik selama pelatihan
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Melatih model
model.fit(X_train_lstm, y_train, validation_data=(X_test_lstm, y_test), epochs=50, batch_size=16, callbacks=[checkpoint])

# Evaluasi performa model
accuracy = model.evaluate(X_test_lstm, y_test)[1]
print("Akurasi pada data uji:", accuracy)

# Simpan model terbaik untuk digunakan pada data baru
best_model = load_model('best_model.h5')


Epoch 1/50
1/5 [=====>........................] - ETA: 22s - loss: 1.3863 - accuracy: 0.1250
Epoch 1: val_accuracy improved from -inf to 0.30000, saving model to best_model.h5
5/5 [==============================] - 7s 410ms/step - loss: 1.3869 - accuracy: 0.1625 - val_loss: 1.3862 - val_accuracy: 0.3000
Epoch 2/50
1/5 [=====>........................] - ETA: 0s - loss: 1.3865 - accuracy: 0.0625
Epoch 2: val_accuracy did not improve from 0.30000
5/5 [==============================] - 0s 20ms/step - loss: 1.3865 - accuracy: 0.1500 - val_loss: 1.3867 - val_accuracy: 0.1500
Epoch 3/50
1/5 [=====>........................] - ETA: 0s - loss: 1.3856 - accuracy: 0.3125
Epoch 3: val_accuracy did not improve from 0.30000
5/5 [==============================] - 0s 19ms/step - loss: 1.3862 - accuracy: 0.2750 - val_loss: 1.3871 - val_accuracy: 0.1500
Epoch 4/50
1/5 [=====>........................] - ETA: 0s - loss: 1.3859 - accuracy: 0.2500
Epoch 4: val_accuracy did not improve from 0.30000
5/5 [=====